In [2]:
import requests
import json
import openai
  
openai.api_key = open(".tokens/openai", "r").read()
print("Started")

Started


In [3]:
# Uses anki-connect to connect to anki
response = requests.post('http://localhost:8765', data='{"action": "deckNames", "version": 6}', headers={'Content-type': 'application/json'})
print(response.json())

{'result': ['A2_Wortliste_Goethe', 'B1_Wortliste_DTZ_Goethe', 'Deutsch B1', 'Deutsch B1+', 'DW Nicos Weg B1', 'Standard'], 'error': None}


In [4]:
def invoke(method, **params):
    request_data = json.dumps({
        'action': method,
        'params': params,
        'version': 6
    })

    response = requests.post('http://localhost:8765', data=request_data)
    response_data = json.loads(response.text)

    if response_data.get('error'):
        raise Exception(f"Error: {response_data['error']}")
    else:
        return response_data['result']

In [5]:
reviewed_card_ids = invoke('findCards', query=f'rated:1')
cards = invoke('cardsInfo', cards=reviewed_card_ids)

In [8]:
# Unsupported yet
invoke('getReviewsOfCards', cards=['1497484862085'])

Exception: Error: unsupported action

In [53]:
comma_separated_repeated_words = ', '.join([card['fields']['Wort_DE']['value'] for card in cards])
task = "Create a short story in German for level B1 that helps you read the following words: " + comma_separated_repeated_words

In [80]:
task

'Create a short story in German for level B1 that helps you read the following words: aussteigen, Bad, Bahnsteig, beschreiben, Doppelzimmer, dorther, einverstanden sein, Ende, erkältet sein, Fahrplan, Fahrrad, Führung, Frage, freundlich, gefährlich, Grippe, Gespräch, Haar, hinein, hoffen, Kaffee, Jugendherberge, Krimi, Kunde, Land, Lüge, laut, leben, leihen, Mensch, Mensch, Moment, Motor, merken, Müll, Müll, Messe, Museum, Museum, müssen, Mütze, Mütze, nach, mitbringen, mitmachen, mitnehmen, Nachbar, Nachbar, Nachbarin, Nachbarin, Nachricht, Nähe, Nähe, Natur, Natur, natürlich, nebenan, mögen, mögen, neblig, neblig, nehmen, nehmen, offen, offen, oft, nett, nett, Ohr, neu, online, nicht, nicht, nichts, Ordnung, niemand, Ort, Notiz, Paar, Paar, notwendig, packen, Nudel, Nummer, Papier, Papiere, Parfüm, Parfüm, Park, nützlich, Party, Party, Pass, oben, oben, Obst, Obst, passen, Passwort, Projekt, Projekt, Pause, Prospekt, Prospekt, prüfen, prüfen, Prüfung, Prüfung, Pullover, Pullover, Piz

In [55]:
result = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a German Tutor."},
        {"role": "user", "content": task}
    ]
)
content = result.to_dict_recursive()['choices'][0]["message"]["content"]

<OpenAIObject chat.completion id=chatcmpl-73n8pllNufTJMuU5OljdXHkGJQ1ox at 0x1180bc5e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Es ist lange Zeit her, dass ich eine Reise gemacht habe. Doch vor kurzem hatte ich die Gelegenheit, an einer F\u00fchrung in einem Museum teilzunehmen. Es war ein Krimi-Museum und ich bin ein gro\u00dfer Fan von Krimis. Ich musste fr\u00fch aufstehen und packen, bevor ich zur Jugendherberge ging. Der Morgen war neblig und ich musste aufpassen, als ich mit meinem Fahrrad zum Bahnhof fuhr. Zum Gl\u00fcck war ich p\u00fcnktlich auf dem Bahnsteig und konnte meinen Platz im Zug ergattern.\n\nIch hatte ein Doppelzimmer in der Jugendherberge gebucht und mein Nachbar war sehr nett. Bevor ich ausstieg, hatte ich noch ein kurzes Gespr\u00e4ch mit ihm. Er sagte, dass er die Natur in dieser Gegend liebt und oft in der N\u00e4he herumspaziert. Ich wollte ein paar Orte in dieser Gegend besuchen und

In [124]:
test_result = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a German Tutor, generate a test for B1 language level for the following test."},
        {"role": "user", "content": content}
    ]
)
test_content = test_result.to_dict_recursive()['choices'][0]["message"]["content"]

In [125]:
# save to file with the current datetime
import datetime
with open(f"german-text-{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.md", "w") as f:
    f.write("## Task:\n\n")
    f.write(task)
    f.write("\n\n## Story:\n\n")
    f.write(content)
    f.write("\n\n## Test\n")
    f.write(test_content)
